In [21]:
import numpy as np
import pandas as pd
# Data preparation and Evaluation
import os
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
# Models
from sklearn import tree
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, SplineTransformer
# Model evaluation
from sklearn.model_selection import cross_val_score
# Encoders
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
# Dimensionality reduction
from sklearn.decomposition import PCA
# Data visualization
from tabulate import tabulate
import time

np.set_printoptions(precision=2, suppress=True)
np.random.seed(42)

pd.options.display.float_format = '{:.2f}'.format

In [22]:
input_path = "../data/slices/"


# df = pd.read_csv(input_path + "all_volume_components.csv")
# df = pd.read_csv(input_path + "total_volume_only.csv")
# df = df[df["type"] == "organic"]
# df = pd.read_csv(input_path + "all_volume_components_year.csv")
df = pd.read_csv(input_path + "all_volume_components_year_week.csv")

print("Sample from CSV read")
print(tabulate(df.sample(5), headers='keys', tablefmt='psql'))

# Splitting features and target
X, y = df.drop('average_price', axis=1), df['average_price']

Sample from CSV read
+-------+-----------------+-----------+----------+---------+--------------+--------------+---------------+--------------+----------------------+--------+--------+
|       |   average_price |      4046 |     4225 |    4770 |   small_bags |   large_bags |   xlarge_bags | type         | geography            |   year |   week |
|-------+-----------------+-----------+----------+---------+--------------+--------------+---------------+--------------+----------------------+--------+--------|
| 15718 |            1.64 |      0    |   202.17 |    0    |     23598.8  |         0    |             0 | organic      | Northern New England |   2017 |     41 |
| 28607 |            1.17 |   3238.72 | 41880.7  |   14.17 |     24371.6  |      8594.3  |          1810 | conventional | Syracuse             |   2020 |      7 |
| 27406 |            1.85 |   7857    |  7205    |    0    |     39803    |        47    |             0 | organic      | San Francisco        |   2019 |     48 |
|

In [23]:
print("Features shape: ", X.shape)
print("Target shape: ", y.shape)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.33
)

print("Training features shape: ", X_train.shape)
print("Test features shape: ", X_test.shape)

Features shape:  (33045, 10)
Target shape:  (33045,)
Training features shape:  (22140, 10)
Test features shape:  (10905, 10)


In [24]:
numeric_features = ["4046","4225","4770","small_bags","large_bags","xlarge_bags", "year", "week"]

numeric_transformer_no_scaling = Pipeline(
    steps=[
        ("dim reduction", PCA()),
        ("scaler", StandardScaler(with_mean=False, with_std=False)) # Not scaling
    ]
)

categorical_features = ["type", "geography"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder())
    ]
)

decision_tree_preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer_no_scaling, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

decision_tree_pipeline = Pipeline(
    steps=[
        ("preprocessor", decision_tree_preprocessor), 
        ("decision_tree", tree.DecisionTreeRegressor())
    ]
)

decision_tree_pipeline_depth_20 = Pipeline(
    steps=[
        ("preprocessor", decision_tree_preprocessor), 
        ("decision_tree", tree.DecisionTreeRegressor(max_depth=15))
    ]
)

decision_tree_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('dim '
                                                                   'reduction',
                                                                   PCA()),
                                                                  ('scaler',
                                                                   StandardScaler(with_mean=False,
                                                                                  with_std=False))]),
                                                  ['4046', '4225', '4770',
                                                   'small_bags', 'large_bags',
                                                   'xlarge_bags', 'year',
                                                   'week']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['type', 'geography'])])),
                ('decision_tree', DecisionTreeRegressor())])

In [25]:
numeric_features = ["4046","4225","4770","small_bags","large_bags","xlarge_bags"]
numeric_transformer = Pipeline(
    steps=[
        ("dim reduction", PCA()),
        ("scaler", StandardScaler())
        #("polynomial", PolynomialFeatures(2)),
        # ("scaler 2", StandardScaler()) 
    ]
)

categorical_features = ["type", "geography", "year", "week"]
categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder())
    ]
)

linear_regression_processor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

linear_regression_pipeline = Pipeline(
    steps=[
        ("preprocessor", linear_regression_processor), 
        ("linear regression", LinearRegression())
    ]
)

from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV

ridge_regression_pipeline = Pipeline(
    steps=[
        ("preprocessor", linear_regression_processor), 
        ("ridge regression", RidgeCV(alphas = [0.0001, 0.001,0.01, 0.1, 1, 3, 10]))
    ]
)

lasso_regression_pipeline = Pipeline(
    steps=[
        ("preprocessor", linear_regression_processor), 
        ("ridge regression", LassoCV(alphas = [0.0001, 0.001,0.01, 0.1, 1, 3, 10]))
    ]
)

linear_regression_pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('dim '
                                                                   'reduction',
                                                                   PCA()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['4046', '4225', '4770',
                                                   'small_bags', 'large_bags',
                                                   'xlarge_bags']),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['type', 'geography', 'year',
                                                   'week'])])),
                ('linear regression', LinearRegression())])

In [26]:
pipelines = {
    "Decision Tree Max Depth": decision_tree_pipeline, 
    #"Decision Tree Depth=20": decision_tree_pipeline_depth_20,
    "Linear regression": linear_regression_pipeline,
    # "Lasso regression": lasso_regression_pipeline,
    # "Ridge regression": ridge_regression_pipeline
}

for pipeline_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)

In [27]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

models_errors = {}

print("Errors on training data \n")

for pipeline_name, pipeline in pipelines.items():
        pipeline.fit(X_train, y_train)
        y_predicted = pipeline.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_predicted)
        mse = mean_squared_error(y_test, y_predicted)
        rmse = root_mean_squared_error(y_test, y_predicted)
        r2 = r2_score(y_test, y_predicted)
        
        
        errors = {pipeline.steps[1][1]: [mae, mse, rmse, r2]}
        models_errors.update(errors)
        

print("Average unit avocado price on test data:", round(y_test.mean(), 2),"$")

models_errors_df = pd.DataFrame.from_dict(models_errors, orient="index", columns=["MAE", "MSE", "RMSE", "R2"])
models_errors_df



Errors on training data 

Average unit avocado price on test data: 1.38 $


,MAE,MSE,RMSE,R2
DecisionTreeRegressor(),0.12,0.03,0.18,0.78
LinearRegression(),0.18,0.05,0.23,0.62


In [28]:
from sklearn.model_selection import cross_val_score

cv_rmse_scores = {}

for pipeline_name, pipeline in pipelines.items():
    
    pipeline_rmse_scores = -cross_val_score(
        pipeline, 
        X_train,
        y_train,
        scoring="neg_root_mean_squared_error", 
        cv=5
    )   
    
    cv_rmse_scores.update({pipeline.steps[1][1]: pipeline_rmse_scores})

print("Cross validations scores \n")

cv_scores_df = pd.DataFrame.from_dict(cv_rmse_scores, orient="index", columns=["CV1", "CV2", "CV3", "CV4", "CV5"])
cv_scores_df

Cross validations scores 



,CV1,CV2,CV3,CV4,CV5
DecisionTreeRegressor(),0.19,0.19,0.19,0.19,0.19
LinearRegression(),0.23,0.23,0.23,0.24,0.24


In [29]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

df = pd.read_csv("../data/slices/plot.csv")

mask1 = df['location_level'] == "Level 1"
mask2 = df['type'] == 'conventional'

df_plot = df[mask1 & mask2]

fig = make_subplots(rows=1, cols=1, subplot_titles=("Original","Predicted"))

fig.append_trace(go.Scatter(
    x=df_plot['date'],
    y=df_plot['average_price'],
    name="Average Price Real"
), row=1, col=1)


for pipeline_name, pipeline in pipelines.items():
    
    y_predict = pipeline.predict(X)
    df['average_price_predicted'] = y_predict
    
    df_plot = df[mask1 & mask2]

    fig.append_trace(go.Scatter(
        x=df_plot['date'],
        y=df_plot['average_price_predicted'],
        name=f"{pipeline.steps[1][1]}"
    ), row=1, col=1)

fig.update_layout(height=600, title_text="Stacked Subplots")
fig.show()